<a href="https://colab.research.google.com/github/mobarakol/Project_Gen/blob/main/MedGPT_PathVQA_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downlaod code and dataset

In [ ]:
!pip -q install peft accelerate datasets
!pip -q install -i https://pypi.org/simple/ bitsandbytes
!pip -q install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install -q -U evaluate bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00


Downloading PathVQA dataset:(manually)<br>
Downloading PathVQA dataset:(HF)<br>
```
from datasets import load_dataset
import datasets
datasets.config.DOWNLOADED_DATASETS_PATH = "vqa_datasets"
dataset = load_dataset("flaviagiammarino/path-vqa")
train_data = dataset['train']
val_data = dataset['validation']
```

In [ ]:
!mkdir vqa_datasets
!gdown --id 1eDts_DGDrSRIGl7ajQZOip30cBP4J55x
!mv pvqa.zip vqa_datasets
!unzip -q vqa_datasets/pvqa.zip -d vqa_datasets

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1eDts_DGDrSRIGl7ajQZOip30cBP4J55x
From (redirected): https://drive.google.com/uc?id=1eDts_DGDrSRIGl7ajQZOip30cBP4J55x&confirm=t&uuid=9c5d9f08-8e5d-4818-98ac-ec2fac42b81e
To: /content/pvqa.zip
100% 1.80G/1.80G [00:30<00:00, 59.0MB/s]


Preprocessing the PathVQA for GPT training:

In [ ]:
import clip
import torch
import pandas as pd
import pickle
from tqdm import tqdm
from PIL import Image
from transformers import GPT2Tokenizer
import numpy as np

def update_classes(pkl_train, pkl_val, pkl_test):
    # standardize answer ids across datasets and compute the maximum number of generated output tokens based on the train set
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    with open(pkl_train, 'rb') as f:
            data_train = pickle.load(f)
    with open(pkl_val, 'rb') as f:
            data_val = pickle.load(f)
    with open(pkl_test, 'rb') as f:
            data_test = pickle.load(f)

    cur_id = 0
    class_names_list = []
    class_ids_list = [[],[],[]]

    for i, data in enumerate([data_train,data_val,data_test]):

        for answer in data['answers']:
            if answer not in class_names_list:
                class_names_list.append(answer)
                class_ids_list[i].append(cur_id)
                cur_id+=1
            else:
                class_ids_list[i].append(class_names_list.index(answer))
    q_lens = []
    a_lens = []
    for question in data_train['questions']:
        q_lens.append(len(tokenizer.encode(question)))
    for answer in data_train['answers']:
        a_lens.append(len(tokenizer.encode(str(answer))))

    data_train['class_ids'] = class_ids_list[0]
    data_val['class_ids'] = class_ids_list[1]
    data_test['class_ids'] = class_ids_list[2]

    data_train['class_names'] = class_names_list
    data_val['class_names'] = class_names_list
    data_test['class_names'] = class_names_list

    data_train['max_seqs_len']=(int(np.mean(q_lens)+2*np.std(q_lens)),int(np.mean(a_lens)+2*np.std(a_lens)))
    data_val['max_seqs_len']=(int(np.mean(q_lens)+2*np.std(q_lens)),int(np.mean(a_lens)+2*np.std(a_lens)))
    data_test['max_seqs_len']=(int(np.mean(q_lens)+2*np.std(q_lens)),int(np.mean(a_lens)+2*np.std(a_lens)))

    with open(pkl_train, 'wb') as f:
        pickle.dump(data_train,f)
    with open(pkl_val, 'wb') as f:
        pickle.dump(data_val,f)
    with open(pkl_test, 'wb') as f:
        pickle.dump(data_test,f)

def preprocess_pathvqa(split, out_path):
    device = torch.device('cuda:0')
    clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
    data =  pd.read_pickle('vqa_datasets/pvqa/qas/{}/{}_qa.pkl'.format(split,split))
    print("%0d captions loaded from json " % len(data))
    all_img_prefixes = []
    img_ids = []
    img_paths = []
    all_questions = []
    all_answers = []
    img_dict = {}
    for i in tqdm(range(len(data))):
        d = data[i]
        if d['answer']!="yes" and d['answer']!="no":
            img_id = d["image"]
            filename = "vqa_datasets/pvqa/images/{}/{}.jpg".format(split,img_id)
            with torch.no_grad():
                prefix_i = clip_model.encode_image(preprocess(Image.open(filename)).unsqueeze(0).to(device)).cpu()
            if img_id not in img_dict.keys():
                img_dict[img_id] = [[d['question']],[d['answer']],prefix_i,filename]
            else:
                img_dict[img_id][0].append(d['question'])
                img_dict[img_id][1].append(d['answer'])
    # this dictionary is converted into a format that is sutiable for the data loader. Each data point contains a 'img_id', that corresponds is the index of the corresponding
    # CLIP embedding of the image in 'img_prefix'.
    for img_id, imgs in enumerate(img_dict.keys()):
        all_img_prefixes.append(img_dict[imgs][2])
        for q in range(len(img_dict[imgs][0])):
            all_questions.append(img_dict[imgs][0][q])
            all_answers.append(img_dict[imgs][1][q])
            img_ids.append(img_id)
            img_paths.append(img_dict[imgs][3])

    all_data = {"img_prefix": torch.cat(all_img_prefixes, dim=0), "img_ids": img_ids, "questions": all_questions,'answers': all_answers,'img_path': img_paths}

    with open(out_path, 'wb') as f:
        pickle.dump(all_data,f)
    print('Done')
    print("%0d embeddings saved " % len(all_img_prefixes))


for split in ['train','val','test']:
    out_path = "vqa_datasets/pvqa/{}.pkl".format(split)
    preprocess_pathvqa(split,out_path)

update_classes("vqa_datasets/pvqa/train.pkl", "vqa_datasets/pvqa/val.pkl", "vqa_datasets/pvqa/test.pkl")

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 58.6MiB/s]


19755 captions loaded from json 


100%|██████████| 19755/19755 [04:05<00:00, 80.54it/s]


Done
2577 embeddings saved 
6279 captions loaded from json 


100%|██████████| 6279/6279 [01:18<00:00, 80.44it/s]


...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

# Start Training Process

In [ ]:
from tqdm import tqdm
import sys
import torch
import torch.nn as nn
from transformers import set_seed, GPT2Config, GPT2Tokenizer
from transformers import AutoTokenizer
from transformers.models.biogpt import BioGptTokenizer
import os
import pandas as pd
from torch.utils.data import Dataset
import pickle
from torch.utils.data import DataLoader, random_split
import numpy as np
import pdb
import argparse

class medvqaDataset(Dataset):
    def __init__(self, path, split='train',like_test=False,prefix_length=2,model_type = 'gpt2'):
        super().__init__()
        data_path = path+split+'.pkl'
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        sys.stdout.flush()
        self.model_type = model_type
        self.tokenizer = AutoTokenizer.from_pretrained(model_type)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.img_ids = data["img_ids"]
        self.img_prefixes = data["img_prefix"]
        self.questions = data['questions']
        self.answers = data['answers']
        self.img_paths = data['img_path']

        self.max_seqs_len = data['max_seqs_len']
        self.labels = data['class_ids']
        self.train_setting = True if (split!='test'and like_test==False) else False
        self.prefix_len = prefix_length

    def __len__(self):
        return len(self.answers)
    def pad_sequences(self,index):
        m = [torch.tensor(self.tokenizer.encode('question: ')),torch.tensor(self.tokenizer.encode(' context:')),torch.tensor(self.tokenizer.encode('answer ')),torch.tensor(self.tokenizer.encode('<|endoftext|>'))]
        m_mask = [torch.ones(len(self.tokenizer.encode('question: '))),torch.ones(len(self.tokenizer.encode(' context:'))),torch.ones(len(self.tokenizer.encode('answer '))),torch.zeros(len(self.tokenizer.encode('<|endoftext|>')))]

        if self.train_setting:
            # construct the model input. The order is question, image, answer. During training the answer is masked. Any padding is placed on the right of the sequence.
            # placeholder tokens are used on the location where the visual prefix will be inserted, with q_len indicating this location.
            q=torch.tensor(self.tokenizer.encode(self.questions[index]))
            a=torch.tensor(self.tokenizer.encode(str(self.answers[index])))

            q,q_mask,leftover_tokens = self.make_padding(self.max_seqs_len[0],q,question=True)
            q_len = m[0].size(0) + q.size(0) + m[1].size(0)
            a,a_mask,_ = self.make_padding(self.max_seqs_len[1],a,leftover_tokens=leftover_tokens)
            if len((a==0).nonzero())!=0:
                pad_start = (a==0).nonzero()[0]
            else:
                pad_start=[]
            a = torch.cat((a,m[3])) if len(pad_start)==0 else torch.cat((a[:pad_start],m[3],a[pad_start:]))
            q = torch.cat((m[0],q,m[1],torch.ones(self.prefix_len),m[2],a))

            q_mask = torch.cat((m_mask[0],q_mask,m_mask[1],torch.ones(self.prefix_len),m_mask[2],a_mask,m_mask[3]))
            return q,q_mask, q_len
        else:
            # in the test stage we do not have acces to the answer, so we just load the question.
            # since inference is not performed batch-wised we don't need to apply padding
            q = torch.tensor(self.tokenizer.encode(self.questions[index]))

            q,q_mask,_ = self.make_padding_test_setting(self.max_seqs_len[0],q)
            q_len = m[0].size(0) + q.size(0) + m[1].size(0)
            q = torch.cat((m[0],q,m[1],torch.ones(self.prefix_len),m[2]))


            q_mask = torch.cat((m_mask[0],q_mask,m_mask[1]))
            return q,q_mask,q_len

    def make_padding(self, max_len, tokens, question=False,leftover_tokens=0):
        padding = max_len - tokens.size(0)
        if padding > 0:
            if question:
                leftover_tokens = padding
                mask = torch.ones(tokens.size(0))
            else:
                tokens = torch.cat((tokens, torch.zeros(padding+leftover_tokens)))
                mask = torch.zeros(max_len+leftover_tokens)

        elif padding==0:
            if question:
                mask = torch.ones(tokens.size(0))
            else:
                mask = torch.zeros(tokens.size(0)+leftover_tokens)
                tokens = torch.cat((tokens,torch.zeros(leftover_tokens)))


        elif padding < 0:
            if question:
                tokens = tokens[:max_len]
                mask = torch.ones(max_len)
            else:
                tokens = torch.cat((tokens[:max_len], torch.zeros(leftover_tokens)))
                mask = torch.zeros(max_len+ leftover_tokens)
        return tokens, mask, leftover_tokens
    def make_padding_test_setting(self, max_len, tokens,do_padding=False):
        padding = max_len - tokens.size(0)
        padding_len = 0
        if padding > 0:
            if do_padding:
                mask = torch.cat((torch.ones(tokens.size(0)),torch.zeros(padding)))
                tokens = torch.cat((tokens,torch.zeros(padding)))
                padding_len = padding
            else:
                mask = torch.ones(tokens.size(0))
        elif padding ==0:
            mask = torch.ones(max_len)
        elif padding < 0:
            tokens = tokens[:max_len]
            mask = torch.ones(max_len)
        return tokens, mask, padding_len

    def __getitem__(self, index):
        prefix = self.img_prefixes[self.img_ids[index]]
        tokens, mask, q_len  = self.pad_sequences(index)
        return prefix,  self.labels[index], tokens, mask, q_len



Restart the session, please:

In [ ]:
import torch
import torch.nn as nn
from typing import Tuple
import sys
import transformers
from transformers import set_seed, GPT2Config, GPT2Tokenizer, GPT2LMHeadModel
from transformers.models.biogpt import BioGptForCausalLM, BioGptTokenizer, BioGptConfig
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig
from peft import LoraConfig, get_peft_model,get_peft_config,PeftModelForCausalLM,TaskType,PrefixTuningConfig, PromptEncoderConfig, PromptTuningConfig
import random

#eval
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score,roc_auc_score
# from utils import generate_beam
from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer
import pdb
from evaluate import load
import collections
from torch.cuda.amp import autocast
import os

class MLP(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(nn.Dropout(p=0.5))
                layers.append(act())
        self.model = nn.Sequential(*layers)

class VQAmedModel(nn.Module):
    def __init__(
        self,
        prefix_length=2,
        clip_length=2,
        prefix_size=512,
        num_layers=8,
        setting="lora",
        mapping_type="MLP",
        args=None,
    ):
        super(VQAmedModel, self).__init__()
        gpttype = args.model_type
        self.gpttype = gpttype
        self.model_type = gpttype
        self.setting = setting
        self.prefix_length = prefix_length
        self.gpt = AutoModelForCausalLM.from_pretrained(gpttype,load_in_8bit=True,device_map='auto')
        # load the relevant fine-tuning strategy
        if setting == "lora":
            peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
            self.gpt = get_peft_model(self.gpt,peft_config)
        elif setting=="prefixtuning":
            peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
            self.gpt = get_peft_model(self.gpt,peft_config)
        elif setting=="p_tuning":
            peft_config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
            self.gpt = get_peft_model(self.gpt,peft_config)
        elif setting=="prompttuning":
            peft_config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
            self.gpt = get_peft_model(self.gpt,peft_config)
        elif setting=='frozen':
            for param in self.gpt.transformer.parameters():
                param.requires_grad = False
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpttype)
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if mapping_type == "MLP":
            self.clip_project = MLP((
                    prefix_size,
                    (self.gpt_embedding_size * prefix_length) // 2,
                    self.gpt_embedding_size * prefix_length,
                    self.gpt_embedding_size * prefix_length))
        # elif mapping_type == "Transformer":
        #     self.clip_project = TransformerMapper(
        #         prefix_size,
        #         self.gpt_embedding_size,
        #         prefix_length,
        #         clip_length,
        #         num_layers)
        else:
            raise ValueError("select valid mapping type: MLP or Transformer")

    def forward(self, prefix, labels, tokens, mask, q_len, batch_size):
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        if self.gpttype=='microsoft/biogpt':
            embedding = self.gpt.transformer.embed_tokens(tokens)
        else:
            embedding = self.gpt.transformer.wte(tokens)
        for b in range(batch_size):
            # insert the visual prefix after the question
            embedding[b,q_len[b]:q_len[b]+self.prefix_length,:] = prefix_projections[b]
        return self.gpt(inputs_embeds=embedding, attention_mask=mask)
    def generate(self, prefix, labels, tokens, mask, q_len):
        prefix_projections = self.clip_project(prefix.view(1, -1)).view(self.prefix_length, self.gpt_embedding_size)
        if self.gpttype=='microsoft/biogpt':
            embedding_txt = self.gpt.transformer.embed_tokens(tokens)
        else:
            embedding_txt = self.gpt.transformer.wte(tokens)
        embedding_txt[q_len:q_len+self.prefix_length,:] = prefix_projections
        return embedding_txt


def set_random_seeds(random_seed=0):

    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)



import torch
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
def treebank_tokenize(s):
    return TreebankWordTokenizer().tokenize(s)
def generate_beam(
    model,
    tokenizer,
    beam_size: int = 5,
    generated=None,
    entry_length=65,
    temperature=1.0,
    stop_token: str = "<|endoftext|>",
):
    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits

            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

            logits = logits.softmax(-1).log()
            # final_logit

            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(
                    beam_size, -1
                )
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            if model.model_type == "biogpt":
                next_token_embed = model.gpt.biogpt.embed_tokens(
                    next_tokens.squeeze()
                ).view(generated.shape[0], 1, -1)
            elif model.model_type == "gpt2":
                next_token_embed = model.gpt.transformer.wte(
                    next_tokens.squeeze()
                ).view(generated.shape[0], 1, -1)
            else:
                next_token_embed = model.gpt.get_input_embeddings()(tokens[:,-1])
                next_token_embed=next_token_embed.squeeze().view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [
        tokenizer.decode(output[: int(length)])
        for output, length in zip(output_list, seq_lengths)
    ]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score,roc_auc_score
# from utils import generate_beam
from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer
import pdb
from evaluate import load
import collections
from torch.cuda.amp import autocast
import os

def eval_gpt_open_ended(model, dataset, args, print_vis_token_meaning=False):
    model.eval()
    model=model.cuda()
    bert_score = load("bertscore")
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained(model.model_type)
    bleu_avg1=0.
    bert_avg1 = 0.
    bert_avg2 = 0.
    bert_avg3 = 0.
    f1_avg = 0.
    acc = 0.
    acc_oe = 0.
    acc_yn = 0.
    c_oe =1e-9
    c_yn =1e-9
    with tqdm(total=len(dataset)) as epoch_pbar:
        epoch_pbar.set_description("Testing")
        for item in range(len(dataset)):
            prefix,  labels, tokens, mask, q_len = dataset[item]
            prefix = prefix.type(torch.float32).cuda()
            tokens = tokens.type(torch.long).cuda()
            mask = mask.cuda()
            with autocast(dtype=torch.float16):
              with torch.no_grad():
                  embed = model.generate(prefix,labels,tokens,mask,q_len).view(1,tokens.size(0),-1)
                  if print_vis_token_meaning:
                    prefix_projections = embed[:,q_len:q_len+model.prefix_length,:]
                    for i in range(prefix_projections.size(1)):
                      print_nearest_text_token(prefix_projections[0,i], model)
                  out_text = generate_beam(model, model.tokenizer,generated=embed,entry_length=dataset.max_seqs_len[1], temperature=1)[0]

            if out_text.lower()==dataset.answers[item].lower():
              acc+=1
            if dataset.answers[item].lower()=='yes' or dataset.answers[item].lower()=='no':
              if out_text.lower()==dataset.answers[item].lower():
                acc_yn+=1
              c_yn+=1
            else:
              if out_text.lower()==dataset.answers[item].lower():
                acc_oe+=1
              c_oe+=1

            reference = [str(dataset.answers[item])]
            candidate = [out_text]

            bleu_1 = sentence_bleu(reference[0], candidate[0], weights=(1, 0, 0, 0))

            a = bert_score.compute(references = reference,predictions = candidate,model_type = 'bert-base-uncased')
            bert_avg1+= a['precision'][0]
            bert_avg2+= a['recall'][0]
            bert_avg3+= a['f1'][0]


            f1_avg += compute_f1(tokenizer.encode(reference[0]),tokenizer.encode(candidate[0]))
            bleu_avg1+=bleu_1


    print('------------')
    print("BLEU {}".format(round(bleu_avg1/len(dataset),3)))
    print("BERTScore {}".format(round(bert_avg3/len(dataset),3)))
    print("F1 {}".format(round(f1_avg/len(dataset),3)))
    print("Accuracy {}".format(round(acc/len(dataset),3)))
    print("Accuracy YN{}".format(round(acc_yn/c_yn,3)))
    print("Accuracy OE{}".format(round(acc_oe/c_oe,3)))

def print_nearest_text_token(vis_token, model):
    """print the nearest token in the vocabulary to the given token through model.gpt.embeddings.weight"""
    embeddings = model.gpt.transformer.wte.weight
    distances = torch.norm(embeddings - vis_token, dim=1)
    nearest_token_idx = torch.argmin(distances)
    print(model.tokenizer.decode([nearest_token_idx.item()]))

def compute_f1(gold_toks, pred_toks):
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1


def pytorch_model_run(train_loader, valid_loader, test_dataset, model_obj, args):
    accelerator = Accelerator()
    device = accelerator.device

    if not os.path.exists(args.out_dir):
        os.makedirs(args.out_dir)

    model = model_obj.to(device)

    optimizer = AdamW(model.parameters(), lr=args.lr)


    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=args.warmup_steps,
        num_training_steps=args.epochs * len(train_loader),
    )

    ## 💡 introduce all components to accelerate library
    model, optimizer, train_loader, scheduler = accelerator.prepare(
        model, optimizer, train_loader, scheduler
    )
    valid_loader = accelerator.prepare(valid_loader)


    best_valid_loss = float("inf")
    counter = 0
    n_epochs = args.epochs
    accelerator.wait_for_everyone()
    for epoch in range(args.epochs):
        model.to(device)
        with tqdm(total=args.batch_size * len(train_loader)) as epoch_pbar:
            epoch_pbar.set_description(f"Epoch {epoch}")
            start_time = time.time()
            model.train()
            total_loss = 0.0

            for i, (prefix, labels, tokens, mask, q_len) in enumerate(train_loader):
                with accelerator.accumulate(model):
                    prefix = prefix.type(torch.float32)
                    tokens = tokens.type(torch.long)
                    mask = mask.type(torch.long)
                    q_len = q_len.type(torch.long)
                    outputs = model(prefix, labels, tokens, mask, q_len, batch_size=args.batch_size)
                    logits = outputs.logits
                    loss = 0.

                    shift = 10 if args.setting=="p_tuning" or args.setting=="prompttuning" else 0

                    for b in range(logits.size(0)):
                        condensed_tokens = tokens[b,q_len[b]+model.prefix_length+1:]
                        condensed_logits = logits[b,shift+q_len[b]+model.prefix_length:-1]

                        loss+= nnf.cross_entropy(condensed_logits.reshape(-1,logits.shape[-1]), condensed_tokens.flatten(), ignore_index=0)
                    loss=loss/logits.size(0)

                    accelerator.backward(loss)
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    total_loss += loss.item()
                    avg_loss = total_loss / (i+1)
                    desc = f"Epoch {epoch} - loss {avg_loss:.20f}"
                    epoch_pbar.set_description(desc)
                    epoch_pbar.update(prefix.shape[0])
        model.eval()

        total_loss = 0.0
        with tqdm(total=args.batch_size * len(valid_loader)) as epoch_pbar:
            epoch_pbar.set_description(f"VAL Epoch {epoch}")
            for i, (prefix, labels, tokens, mask,q_len) in enumerate(valid_loader):
                torch.cuda.empty_cache()
                prefix = prefix.type(torch.float32)
                tokens = tokens.type(torch.long)
                mask = mask.type(torch.long)
                q_len = q_len.type(torch.long)

                with torch.no_grad():
                    outputs = model(prefix, labels, tokens, mask, q_len, batch_size=args.batch_size)
                    logits = outputs.logits
                    loss = 0.
                    shift = 10 if args.setting=="p_tuning" or args.setting=="prompttuning" else 0
                    for b in range(logits.size(0)):
                        condensed_tokens = tokens[b,q_len[b]+model.prefix_length+1:]
                        condensed_logits = logits[b,shift+q_len[b]+model.prefix_length:-1]
                        loss+= nnf.cross_entropy(condensed_logits.reshape(-1,logits.shape[-1]), condensed_tokens.flatten(), ignore_index=0)
                    loss=loss/logits.size(0)
                    total_loss += loss.item()
                avg_val_loss = total_loss / (i + 1)
                desc = f"VAL Epoch {epoch} - loss {avg_val_loss:.20f}"
                epoch_pbar.set_description(desc)
                epoch_pbar.update(prefix.shape[0])

        if avg_val_loss < best_valid_loss:
            best_valid_loss = avg_val_loss

            torch.save(model.state_dict(), os.path.join(args.out_dir, f"open_ended_latest.pt"))

        scheduler.step()
        elapsed_time = time.time() - start_time
        print(
            "VAL epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s".format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time
            )
        )
        if avg_val_loss > avg_loss:
            counter += 1
        if counter == 5:
            break

        eval_gpt_open_ended(model, test_dataset, args, print_vis_token_meaning=False)
    return model

In [ ]:
import time
import torch.nn.functional as nnf
from accelerate import Accelerator
import os
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
)

def parse_argument():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_type", type=str, default="gpt2-xl", choices=("gpt2-xl", "microsoft/biogpt","stanford-crfm/BioMedLM"))
    parser.add_argument("--setting", type=str, default="frozen", choices=("lora", "frozen",'prefixtuning',"p_tuning","prompttuning", "unfrozen"))
    parser.add_argument("--ablation", type=str, default="none", choices=("remove_question", "remove_visual",'replace_visual',"swap"))
    parser.add_argument("--mapping_type", type=str, default="MLP")
    parser.add_argument("--prefix_length", type=int, default=8)
    parser.add_argument("--dataset_path", type=str, default="vqa_datasets/")
    parser.add_argument("--batch_size", type=int, default=20)
    parser.add_argument("--epochs", type=int, default=30)
    parser.add_argument("--dataset", type=str, default='pathvqa', choices=('pathvqa', 'ovqa', 'slake'))
    parser.add_argument("--lr", type=float, default=1e-4)
    parser.add_argument("--warmup_steps", type=int, default=600)
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--iters_to_accumulate", type=int, default=4)
    parser.add_argument("--validation_step", type=int, default=1000)
    parser.add_argument("--out_dir", default="checkpoints")
    parser.add_argument("--checkpoint", type=str)
    parser.add_argument("--eval", dest="eval", action="store_true")
    parser.add_argument("--verbose", dest="verbose", action="store_true")

    if 'ipykernel' in sys.modules:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()

    set_random_seeds(args.seed)
    return args

args = parse_argument()

suffix = f"v5_prefixlength_{args.prefix_length}_mapping_{args.mapping_type}_seed_{args.seed}_gpttype_{args.model_type.replace('/','')}_setting_{args.setting}_dataset_{args.dataset}"

args.out_dir = os.path.join('checkpoints', suffix)

train_dataset = medvqaDataset(args.dataset_path+'pvqa/',split="train",prefix_length=args.prefix_length,model_type=args.model_type)#,abl=args.ablation)
val_dataset = medvqaDataset(args.dataset_path+'pvqa/',split="val",prefix_length=args.prefix_length,model_type=args.model_type)#, abl=args.ablation)
test_dataset = medvqaDataset(args.dataset_path+'pvqa/',split="test",prefix_length=args.prefix_length,model_type=args.model_type,like_test=True)
print('Sample: Tain={}, valid={}, Test={}'.format(len(train_dataset), len(val_dataset), len(test_dataset)))

train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)

model = VQAmedModel(
            prefix_length=args.prefix_length,
            clip_length=4,
            setting=args.setting,
            mapping_type=args.mapping_type,
            args=args,
        )

model = pytorch_model_run(train_dataloader, val_dataloader, test_dataset, model, args)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Testing:   0%|          | 0/3370 [03:28<?, ?it/s]


KeyboardInterrupt: 

Inference

In [ ]:


def eval_gpt_open_ended(model, dataset, args, print_vis_token_meaning=False):
    model.eval()
    model=model.cuda()
    bert_score = load("bertscore")
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained(model.model_type)
    bleu_avg1=0.
    bert_avg1 = 0.
    bert_avg2 = 0.
    bert_avg3 = 0.
    f1_avg = 0.
    acc = 0.
    acc_oe = 0.
    acc_yn = 0.
    c_oe =1e-9
    c_yn =1e-9
    with tqdm(total=len(dataset)) as epoch_pbar:
        epoch_pbar.set_description("Testing")
        for item in range(len(dataset)):
            prefix,  labels, tokens, mask, q_len = dataset[item]
            prefix = prefix.type(torch.float32).cuda()
            tokens = tokens.type(torch.long).cuda()
            mask = mask.cuda()
            with autocast(dtype=torch.float16):
              with torch.no_grad():
                  embed = model.generate(prefix,labels,tokens,mask,q_len).view(1,tokens.size(0),-1)
                  if print_vis_token_meaning:
                    prefix_projections = embed[:,q_len:q_len+model.prefix_length,:]
                    for i in range(prefix_projections.size(1)):
                      print_nearest_text_token(prefix_projections[0,i], model)
                  out_text = generate_beam(model, model.tokenizer,generated=embed,entry_length=dataset.max_seqs_len[1], temperature=1)[0]

            if out_text.lower()==dataset.answers[item].lower():
              acc+=1
            if dataset.answers[item].lower()=='yes' or dataset.answers[item].lower()=='no':
              if out_text.lower()==dataset.answers[item].lower():
                acc_yn+=1
              c_yn+=1
            else:
              if out_text.lower()==dataset.answers[item].lower():
                acc_oe+=1
              c_oe+=1

            reference = [str(dataset.answers[item])]
            candidate = [out_text]

            bleu_1 = sentence_bleu(reference[0], candidate[0], weights=(1, 0, 0, 0))

            a = bert_score.compute(references = reference,predictions = candidate,model_type = 'bert-base-uncased')
            bert_avg1+= a['precision'][0]
            bert_avg2+= a['recall'][0]
            bert_avg3+= a['f1'][0]


            f1_avg += compute_f1(tokenizer.encode(reference[0]),tokenizer.encode(candidate[0]))
            bleu_avg1+=bleu_1


    print('------------')
    print("BLEU {}".format(round(bleu_avg1/len(dataset),3)))
    print("BERTScore {}".format(round(bert_avg3/len(dataset),3)))
    print("F1 {}".format(round(f1_avg/len(dataset),3)))
    print("Accuracy {}".format(round(acc/len(dataset),3)))
    print("Accuracy YN{}".format(round(acc_yn/c_yn,3)))
    print("Accuracy OE{}".format(round(acc_oe/c_oe,3)))

# model = VQAmedModel(
#             prefix_length=args.prefix_length,
#             clip_length=4,
#             setting=args.setting,
#             mapping_type=args.mapping_type,
#             args=args,
#         )

checkpoint = os.path.join(args.out_dir, f"open_ended_latest.pt")
if args.verbose:
    print(f">> Loading pre-trained model {checkpoint}!")
if os.path.exists(checkpoint):
    print('ckpt exist')
    model.cuda()
    model.load_state_dict(
        torch.load(checkpoint, map_location=torch.device("cuda")), strict=False
    )
else:
    raise ValueError("Please provide valid path for loading checkpoint")
# eval_gpt_open_ended(model, test_dataset, args, print_vis_token_meaning=False)

ckpt exist


<ipython-input-6-282d88f637b8>:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(checkpoint, map_location=torch.device("cuda")), strict=False
